In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:48707")
client

Client Scheduler: tcp://127.0.0.1:48707 Dashboard: http://127.0.0.1:44931/status,Cluster Workers: 6 Cores: 30 Memory: 127.00 GB


In [2]:
import numpy as np
import xarray as xr
import pandas as pd
from dask.diagnostics import ProgressBar
import os.path as op
import os
import xrft
import gsw
import time
from scipy.interpolate import PchipInterpolator, interp1d
from xgcm.grid import Grid
from xmitgcm import open_mdsdataset
from MITgcmutils import jmd95
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from xlayers import finegrid, layers
from xlayers.core import layers_numpy, layers_apply

In [4]:
ddir = '/tank/chaocean/'
savedir = '/tank/topog/tuchida/TWA/'

In [5]:
grav = 9.81
nensembs = 5
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
Cp = 3.994e3
ySstart = -15
yNstart = 10
yend = 50

xchunk = 250
ychunk = 400
membchunk = 1
dnf = 2160

In [6]:
for nmemb in range(24,36):
    if nmemb == 24:
        dsnap = open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run1963/ocn/' % nmemb), 
                                grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
                                prefix=['diag_ocnSnap',
                                        'diag_Tbgt2D','diag_Tbgt3D',
                                        'diag_Sbgt2D','diag_Sbgt3D'], delta_t=2e2,
#                                 iters=range(943920-2160,943920+2160,2160),
                                iters=943920, 
                                chunks={'XC':xchunk,'XG':xchunk}
                                ).isel(time=-1)
#         dsave = open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run1963/ocn/' % nmemb), 
#                                 grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
#                                 prefix=['diag_ocnTave','diag_ocnSurf'], delta_t=2e2,
#         #                              iters=range(788400+2160,943920+2160,2160)
#                                 iters=943920, chunks={'XC':xchunk,'XG':xchunk}
#                                 ).isel(time=-1)
    else:
        dsnap = xr.concat([dsnap, open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run1963/ocn/' 
                                                          % nmemb), 
                                                  grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
                                                  prefix=['diag_ocnSnap',
                                                          'diag_Tbgt2D','diag_Tbgt3D',
                                                          'diag_Sbgt2D','diag_Sbgt3D'], delta_t=2e2,
#                                                   iters=range(943920-2160,943920+2160,2160),
                                                  iters=943920, 
                                                  chunks={'XC':xchunk,'XG':xchunk}
                                                 ).isel(time=-1)],
                          dim='nmemb')
#         dsave = xr.concat([dsave, open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run1963/ocn/' 
#                                                           % nmemb), 
#                                                   grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
#                                                   prefix=['diag_ocnTave','diag_ocnSurf'], delta_t=2e2,
#         #                                             iters=range(788400+2160,943920+2160,2160)
#                                                   iters=943920, chunks={'XC':xchunk,'XG':xchunk}
#                                                  ).isel(time=-1)],
#                           dim='nmemb')
dsnap

<xarray.Dataset>
Dimensions:   (XC: 1000, XG: 1000, YC: 900, YG: 900, Z: 46, Zl: 46, Zp1: 47, Zu: 46, nmemb: 12)
Coordinates:
    hFacC     (Z, YC, XC) >f4 dask.array<chunksize=(46, 900, 250), meta=np.ndarray>
    PHrefF    (Zp1) >f4 dask.array<chunksize=(47,), meta=np.ndarray>
    dxG       (YG, XC) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    rAz       (YG, XG) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    maskInW   (YC, XG) bool dask.array<chunksize=(900, 250), meta=np.ndarray>
    drF       (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    dyC       (YG, XC) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    maskC     (Z, YC, XC) bool dask.array<chunksize=(46, 900, 250), meta=np.ndarray>
    dxC       (YC, XG) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    time      float64 1.888e+08
    rAs       (YG, XC) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    maskInC   (YC, XC) bool dask.array<chunksize=(900, 250), meta=np.ndarray>
    PHrefC    (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    maskW     (Z, YC, XG) bool dask.array<chunksize=(46, 900, 250), meta=np.ndarray>
    maskInS   (YG, XC) bool dask.array<chunksize=(900, 250), meta=np.ndarray>
    hFacS     (Z, YG, XC) >f4 dask.array<chunksize=(46, 900, 250), meta=np.ndarray>
    dyG       (YC, XG) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    iter      int64 dask.array<chunksize=(), meta=np.ndarray>
    drC       (Zp1) >f4 dask.array<chunksize=(47,), meta=np.ndarray>
    maskS     (Z, YG, XC) bool dask.array<chunksize=(46, 900, 250), meta=np.ndarray>
    rAw       (YC, XG) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    Depth     (YC, XC) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    rA        (YC, XC) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    hFacW     (Z, YC, XG) >f4 dask.array<chunksize=(46, 900, 250), meta=np.ndarray>
  * YC        (YC) >f4 -19.916634 -19.8333 -19.749968 ... 54.916702 55.000034
  * Zp1       (Zp1) >f4 0.0 -6.0935454 -12.814552 ... -5749.9097 -6000.3726
  * Z         (Z) >f4 -3.0467727 -9.454049 -16.363966 ... -5624.9517 -5875.141
  * YG        (YG) >f4 -19.9583 -19.874968 -19.791634 ... 54.875034 54.95837
  * Zl        (Zl) >f4 0.0 -6.0935454 -12.814552 ... -5499.9937 -5749.9097
  * XC        (XC) >f4 262.00003 262.0861 262.17215 ... 347.88644 347.9725
  * Zu        (Zu) >f4 -6.0935454 -12.814552 -19.91338 ... -5749.9097 -6000.3726
  * XG        (XG) >f4 261.957 262.04306 262.12912 ... 347.8434 347.92947
Dimensions without coordinates: nmemb
Data variables:
    TFLUX     (nmemb, YC, XC) float32 dask.array<chunksize=(1, 900, 250), meta=np.ndarray>
    oceQsw    (nmemb, YC, XC) float32 dask.array<chunksize=(1, 900, 250), meta=np.ndarray>
    WTHMASS   (nmemb, YC, XC) float32 dask.array<chunksize=(1, 900, 250), meta=np.ndarray>
    TOTSTEND  (nmemb, Z, YC, XC) float32 dask.array<chunksize=(1, 46, 900, 250), meta=np.ndarray>
    ADVr_SLT  (nmemb, Zl, YC, XC) float32 dask.array<chunksize=(1, 46, 900, 250), meta=np.ndarray>
    ADVx_SLT  (nmemb, Z, YC, XG) float32 dask.array<chunksize=(1, 46, 900, 250), meta=np.ndarray>
    ADVy_SLT  (nmemb, Z, YG, XC) float32 dask.array<chunksize=(1, 46, 900, 250), meta=np.ndarray>
    Diss_SLT  (nmemb, Z, YC, XC) float32 dask.array<chunksize=(1, 46, 900, 250), meta=np.ndarray>
    SFLUX     (nmemb, YC, XC) float32 dask.array<chunksize=(1, 900, 250), meta=np.ndarray>
    oceFWflx  (nmemb, YC, XC) float32 dask.array<chunksize=(1, 900, 250), meta=np.ndarray>
    WSLTMASS  (nmemb, YC, XC) float32 dask.array<chunksize=(1, 900, 250), meta=np.ndarray>
    TOTTTEND  (nmemb, Z, YC, XC) float32 dask.array<chunksize=(1, 46, 900, 250), meta=np.ndarray>
    ADVr_TH   (nmemb, Zl, YC, XC) float32 dask.array<chunksize=(1, 46, 900, 250), meta=np.ndarray>
    ADVx_TH   (nmemb, Z, YC, XG) float32 dask.array<chunksize=(1, 46, 900, 250), meta=np.ndarray>
    ADVy_TH   (nmemb, Z, YG, XC) float32 dask.array<chunksize=(1, 46, 900

In [7]:
grid = Grid(dsnap, periodic=['X'])
grid

<xgcm.Grid>
X Axis (periodic):
  * center   XC --> left
  * left     XG --> center
Z Axis (not periodic):
  * center   Z --> left
  * left     Zl --> center
  * outer    Zp1 --> center
  * right    Zu --> center
Y Axis (not periodic):
  * center   YC --> left
  * left     YG --> center

In [8]:
fG = xr.apply_ufunc(gsw.f, dsnap.YG, dask='parallelized')
fC = xr.apply_ufunc(gsw.f, dsnap.YC, dask='parallelized')

In [9]:
rho_snap = xr.apply_ufunc(jmd95.densjmd95, dsnap.SALT, dsnap.THETA,
                          (dsnap.PHIHYD-grav*dsnap.Z)*rhoConst*1e-4, # pressure in [dbar]!!!!
                          dask='parallelized', output_dtypes=[float,]
                         ).where(dsnap.maskC!=0.)
rho_snapx = xr.apply_ufunc(jmd95.densjmd95, grid.interp(dsnap.SALT,'X'), 
                           grid.interp(dsnap.THETA,'X'), 
                           grid.interp((dsnap.PHIHYD-grav*dsnap.Z)*rhoConst*1e-4,'X'), 
                           dask='parallelized', output_dtypes=[float,]
                          ).where(dsnap.maskW!=0.)
rho_snapy = xr.apply_ufunc(jmd95.densjmd95, grid.interp(dsnap.SALT,'Y',boundary='fill'), 
                           grid.interp(dsnap.THETA,'Y',boundary='fill'), 
                           grid.interp((dsnap.PHIHYD-grav*dsnap.Z)*rhoConst*1e-4,
                                       'Y',boundary='fill'),
                           dask='parallelized', output_dtypes=[float,]
                          ).where(dsnap.maskS!=0.)

In [10]:
dpdx_snap = (grid.diff(dsnap.PHIHYD.where(dsnap.maskC!=0.) 
                       * grid.interp(dsnap.dyG,'X'),'X'
                      ) * dsnap.rAw**-1
            ).where(dsnap.maskW!=0.)
dpdy_snap = (grid.diff(dsnap.PHIHYD.where(dsnap.maskC!=0.) 
                       * grid.interp(dsnap.dxG,'Y',boundary='fill'),
                       'Y',boundary='fill'
                      ) * dsnap.rAs**-1
            ).where(dsnap.maskS!=0.)

In [11]:
swfrac = .62 * np.exp(dsnap.Zl/.6) + (1-.62) * np.exp(dsnap.Zl/20.)
swfrac1 = .62 * np.exp(dsnap.Zl.shift(Zl=-1)/.6) + (1-.62) * np.exp(dsnap.Zl.shift(Zl=-1)/20.)
Qsw = (dsnap.oceQsw / (rhoConst*Cp) / (dsnap.drF*dsnap.hFacC) 
       * (swfrac - swfrac1).data
      ).where(dsnap.maskC!=0.)
Tflx = ((dsnap.TFLUX - dsnap.oceQsw)
        / (rhoConst*Cp*dsnap.drF[0]*dsnap.hFacC.isel(Z=0))
       ).where(dsnap.maskInC!=0.)
tsurf_corr = 0.                  # linFSConverveTr = F.
Surf_corr = (tsurf_corr - dsnap.WTHMASS[:,0].where(dsnap.maskInC!=0.)
            ) / (dsnap.drF[0]*dsnap.hFacC.isel(Z=0))
DibaT = dsnap.Diss_TH.where(dsnap.maskC!=0.) + Qsw
DibaT = DibaT + xr.concat([xr.DataArray((Tflx+Surf_corr).data.reshape((len(dsnap.nmemb),1,
                                                                       len(dsnap.YC),len(dsnap.XC))),
                                            dims=['nmemb','Z','YC','XC'],
                                            coords={'nmemb':dsnap.nmemb.data,
                                                    'Z':np.array([dsnap.Z[0].data]),
                                                    'YC':dsnap.YC.data,'XC':dsnap.XC.data}),
                               xr.zeros_like(dsnap.Diss_TH.isel(Z=slice(1,None))
                                            ).reset_coords(drop=True)],
                              dim='Z')
# DibaT

# ##################
Sflx = (dsnap.SFLUX
        / (rhoConst*dsnap.drF[0]*dsnap.hFacC.isel(Z=0))
       ).where(dsnap.maskInC!=0.)
ssurf_corr = 0.                  # linFSConverveTr = F.
Surf_corr = (ssurf_corr - dsnap.WSLTMASS[:,0].where(dsnap.maskInC!=0.)
            ) / (dsnap.drF[0]*dsnap.hFacC.isel(Z=0))
DibaS = dsnap.Diss_SLT.where(dsnap.maskC!=0.)
DibaS = DibaS + xr.concat([xr.DataArray((Sflx+Surf_corr).data.reshape((len(dsnap.nmemb),1,
                                                                       len(dsnap.YC),len(dsnap.XC))),
                                            dims=['nmemb','Z','YC','XC'],
                                            coords={'nmemb':dsnap.nmemb.data,
                                                    'Z':np.array([dsnap.Z[0].data]),
                                                    'YC':dsnap.YC.data,'XC':dsnap.XC.data}),
                               xr.zeros_like(dsnap.Diss_SLT.isel(Z=slice(1,None))
                                            ).reset_coords(drop=True)],
                              dim='Z')
DibaS

<xarray.DataArray (nmemb: 12, Z: 46, YC: 900, XC: 1000)>
dask.array<add, shape=(12, 46, 900, 1000), dtype=float32, chunksize=(1, 45, 900, 250), chunktype=numpy.ndarray>
Coordinates:
  * nmemb    (nmemb) int64 0 1 2 3 4 5 6 7 8 9 10 11
    hFacC    (Z, YC, XC) >f4 dask.array<chunksize=(46, 900, 250), meta=np.ndarray>
    drF      (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    maskC    (Z, YC, XC) bool dask.array<chunksize=(46, 900, 250), meta=np.ndarray>
    time     float64 1.888e+08
    maskInC  (YC, XC) bool dask.array<chunksize=(900, 250), meta=np.ndarray>
    PHrefC   (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    iter     int64 dask.array<chunksize=(), meta=np.ndarray>
    Depth    (YC, XC) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
    rA       (YC, XC) >f4 dask.array<chunksize=(900, 250), meta=np.ndarray>
  * YC       (YC) >f4 -19.916634 -19.8333 -19.749968 ... 54.916702 55.000034
  * Z        (Z) >f4 -3.0467727 -9.454049 -16.363966 ... -5624.9517 -5875.141
  * XC       (XC) >f4 262.00003 262.0861 262.17215 ... 347.88644 347.9725

In [12]:
drf_finer, mapindex, mapfact, cellindex = finegrid.finegrid(np.squeeze(dsnap.drF.sel(Z=slice(None,
                                                                                             None))),
                                                            np.squeeze(dsnap.drC.sel(Zp1=slice(None,
                                                                                               None))),
                                                            10)

In [13]:
Dsig = 30
nlayers = 60

siglayers = np.linspace(20,50,nlayers)
func = interp1d(range(1,nlayers+1), siglayers, fill_value='extrapolate')
sigp1layers = func(np.arange(.5,nlayers+1.5,1))

In [14]:
zz, _ = xr.broadcast(dsnap.Z, rho_snap.isel(Z=0,nmemb=0).sel(YC=slice(ystart-12**-1,yend+2*12**-1)))
zzy, _ = xr.broadcast(dsnap.Z, rho_snapy.isel(Z=0,nmemb=0).sel(YG=slice(ystart-12**-1,yend+2*12**-1)))
zzx, _ = xr.broadcast(dsnap.Z, rho_snapx.isel(Z=0,nmemb=0).sel(YC=slice(ystart-12**-1,yend+2*12**-1)))

In [15]:
zdz_snap = xr.DataArray(np.ones((len(dsnap.nmemb),
                                 len(dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1))),
                                 len(dsnap.XC),len(siglayers))),
                        dims=['nmemb','YC','XC','sig'],
                        coords={'nmemb':range(len(dsnap.nmemb),),
                                'YC':dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1)).data,
                                'XC':dsnap.XC.data,'sig':siglayers}
                       ) * np.nan
dzetaF_snap = zdz_snap.copy()

for tt in range(len(dsnap.nmemb)):
    tmp1 = xr.apply_ufunc(layers_numpy, 
                          xr.ones_like(zz).where(dsnap.maskC!=0.).sel(YC=slice(ystart-12**-1,
                                                                               yend+2*12**-1)),
                          (rho_snap.isel(nmemb=tt)-rhoConst).sel(YC=slice(ystart-12**-1,
                                                                          yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized', 
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()

    tmp2 = xr.apply_ufunc(layers_numpy, 
                          zz.chunk({'XC':xchunk}),
                          (rho_snap.isel(nmemb=tt)-rhoConst).sel(YC=slice(ystart-12**-1,
                                                                          yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized', 
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()


    dzetaF_snap[tt] = tmp1.data
    zdz_snap[tt] = tmp2.data

    del tmp1, tmp2


# dzetaF_snap = dzetaF_snap.chunk({'nmemb':1,'sig':10,'XC':xchunk})
# zdz_snap = zdz_snap.chunk({'nmemb':1,'sig':10,'XC':xchunk})

In [16]:
zxdz_snap = xr.DataArray(np.ones((len(dsnap.nmemb),
                                  len(dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1))),
                                  len(dsnap.XG),len(siglayers))),
                         dims=['nmemb','YC','XG','sig'],
                         coords={'nmemb':range(len(dsnap.nmemb),),
                                 'YC':dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1)).data,
                                 'XG':dsnap.XG.data,'sig':siglayers}
                        ) * np.nan
dzetaFx_snap = zxdz_snap.copy()


for tt in range(len(dsnap.nmemb)):
    tmp3 = xr.apply_ufunc(layers_numpy, 
                          xr.ones_like(zzx).chunk({'XG':xchunk}),
                          (rho_snapx.isel(nmemb=tt)-rhoConst).sel(YC=slice(ystart-12**-1,
                                                                           yend+2*12**-1)
                                                                 ).chunk({'XG':xchunk}),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized', 
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()
    

    tmp4 = xr.apply_ufunc(layers_numpy, 
                          zzx.chunk({'XG':xchunk}),
                          (rho_snapx.isel(nmemb=tt)-rhoConst).sel(YC=slice(ystart-12**-1,
                                                                           yend+2*12**-1)
                                                                 ).chunk({'XG':xchunk}),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized', 
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()


    dzetaFx_snap[tt] = tmp3.data
    zxdz_snap[tt] = tmp4.data

    del tmp3, tmp4


# dzetaFx_snap = dzetaFx_snap.chunk({'nmemb':1,'sig':10,'XG':xchunk})
# zxdz_snap = zxdz_snap.chunk({'nmemb':1,'sig':10,'XG':xchunk})

In [17]:
zydz_snap = xr.DataArray(np.ones((len(dsnap.nmemb),
                                  len(dsnap.YG.sel(YG=slice(ystart-12**-1,yend+2*12**-1))),
                                  len(dsnap.XC),len(siglayers))),
                         dims=['nmemb','YG','XC','sig'],
                         coords={'nmemb':range(len(dsnap.nmemb),),
                                 'YG':dsnap.YG.sel(YG=slice(ystart-12**-1,yend+2*12**-1)).data,
                                 'XC':dsnap.XC.data,'sig':siglayers}
                        ) * np.nan
dzetaFy_snap = zydz_snap.copy()


for tt in range(len(dsnap.nmemb)):
    tmp5 = xr.apply_ufunc(layers_numpy, 
                          xr.ones_like(zzy).chunk({'XC':xchunk}),
                          (rho_snapy.isel(nmemb=tt)-rhoConst).sel(YG=slice(ystart-12**-1,
                                                                           yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized', 
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()
    
    tmp6 = xr.apply_ufunc(layers_numpy, 
                          zzy.chunk({'XC':xchunk}),
                          (rho_snapy.isel(nmemb=tt)-rhoConst).sel(YG=slice(ystart-12**-1,
                                                                           yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized', 
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()


    dzetaFy_snap[tt] = tmp5.data
    zydz_snap[tt] = tmp6.data

    del tmp5, tmp6


# dzetaFy_snap = dzetaFy_snap.chunk({'nmemb':1,'sig':10,'XC':xchunk})
# zydz_snap = zydz_snap.chunk({'nmemb':1,'sig':10,'XC':xchunk})

In [18]:
diaTdz = xr.DataArray(np.ones((len(dsnap.nmemb),
                               len(dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1))),
                               len(dsnap.XC),len(siglayers))),
                      dims=['nmemb','YC','XC','sig'],
                      coords={'nmemb':range(len(dsnap.nmemb),),
                              'YC':dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1)).data,
                              'XC':dsnap.XC.data,'sig':siglayers}
                     ) * np.nan
diaSdz = diaTdz.copy()


for tt in range(len(dsnap.nmemb)):
    tmp3 = xr.apply_ufunc(layers_numpy, 
                          DibaT.isel(nmemb=tt).where(dsnap.maskC!=0.).sel(YC=slice(ystart-12**-1,
                                                                                   yend+2*12**-1)
                                                                         ).chunk({'XC':xchunk,'Z':-1}),
                          (rho_snap.isel(nmemb=tt)-rhoConst).sel(YC=slice(ystart-12**-1,yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized',
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()
    
    tmp4 = xr.apply_ufunc(layers_numpy, 
                          DibaS.isel(nmemb=tt).where(dsnap.maskC!=0.).sel(YC=slice(ystart-12**-1,
                                                                                   yend+2*12**-1)
                                                                         ).chunk({'XC':xchunk,'Z':-1}),
                          (rho_snap.isel(nmemb=tt)-rhoConst).sel(YC=slice(ystart-12**-1,yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized',
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()
    
        
    diaTdz[tt] = tmp3.data
    diaSdz[tt] = tmp4.data
    
    del tmp3, tmp4


diaTdz = diaTdz.chunk({'nmemb':1,'sig':10,'XC':xchunk})
diaSdz = diaSdz.chunk({'nmemb':1,'sig':10,'XC':xchunk})

In [19]:
sdz = xr.DataArray(np.ones((len(dsnap.nmemb),
                            len(dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1))),
                            len(dsnap.XC),len(siglayers))),
                   dims=['nmemb','YC','XC','sig'],
                   coords={'nmemb':range(len(dsnap.nmemb),),
                           'YC':dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1)).data,
                           'XC':dsnap.XC.data,'sig':siglayers}
                  ) * np.nan
tdz = sdz.copy()


for tt in range(len(dsnap.nmemb)):
    tmp5 = xr.apply_ufunc(layers_numpy, 
                          dsnap.THETA.isel(nmemb=tt).where(dsnap.maskC!=0.).sel(YC=slice(ystart-12**-1,
                                                                                         yend+2*12**-1)
                                                                               ).chunk({'XC':xchunk}),
                          (rho_snap.isel(nmemb=tt)-rhoConst).sel(YC=slice(ystart-12**-1,yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized',
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()
    
    tmp6 = xr.apply_ufunc(layers_numpy, 
                          dsnap.SALT.isel(nmemb=tt).where(dsnap.maskC!=0.).sel(YC=slice(ystart-12**-1,
                                                                                        yend+2*12**-1)
                                                                              ).chunk({'XC':xchunk}),
                          (rho_snap.isel(nmemb=tt)-rhoConst).sel(YC=slice(ystart-12**-1,yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized',
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()

        
    tdz[tt] = tmp5.data
    sdz[tt] = tmp6.data
    
    del tmp5, tmp6


tdz = tdz.chunk({'nmemb':1,'sig':10,'XC':xchunk})
sdz = sdz.chunk({'nmemb':1,'sig':10,'XC':xchunk})

In [20]:
pxdz_snap = xr.DataArray(np.ones((len(dsnap.nmemb),
                                  len(dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1))),
                                  len(dsnap.XG),len(siglayers))),
                         dims=['nmemb','YC','XG','sig'],
                         coords={'nmemb':range(len(dsnap.nmemb),),
                                 'YC':dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1)).data,
                                 'XG':dsnap.XG.data,'sig':siglayers}
                        ) * np.nan
pydz_snap = xr.DataArray(np.ones((len(dsnap.nmemb),
                                  len(dsnap.YG.sel(YG=slice(ystart-12**-1,yend+2*12**-1))),
                                  len(dsnap.XC),len(siglayers))),
                         dims=['nmemb','YG','XC','sig'],
                         coords={'nmemb':range(len(dsnap.nmemb),),
                                 'YG':dsnap.YG.sel(YG=slice(ystart-12**-1,yend+2*12**-1)).data,
                                 'XC':dsnap.XC.data,'sig':siglayers}
                        ) * np.nan


for tt in range(len(dsnap.nmemb)):
    tmp1 = xr.apply_ufunc(layers_numpy, 
                          dpdx_snap[tt].sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                           ).chunk({'XG':xchunk}),
                          (rho_snapx[tt]-rhoConst).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                                      ).chunk({'XG':xchunk}),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized',
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()
    
    tmp2 = xr.apply_ufunc(layers_numpy, 
                          dpdy_snap[tt].sel(YG=slice(ystart-12**-1,yend+2*12**-1)),
                          (rho_snapy[tt]-rhoConst).sel(YG=slice(ystart-12**-1,yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized',
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()


    pxdz_snap[tt] = tmp1
    pydz_snap[tt] = tmp2
    
    del tmp1, tmp2

    
# pxdz_snap = pxdz_snap.chunk({'nmemb':1,'sig':10,'XG':xchunk})
# pydz_snap = pydz_snap.chunk({'nmemb':1,'sig':10,'XC':xchunk})

In [21]:
udz_snap = xr.DataArray(np.ones((len(dsnap.nmemb),
                                 len(dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1))),
                                 len(dsnap.XG),len(siglayers))),
                        dims=['nmemb','YC','XG','sig'],
                        coords={'nmemb':range(len(dsnap.nmemb),),
                                'YC':dsnap.YC.sel(YC=slice(ystart-12**-1,yend+2*12**-1)).data,
                                'XG':dsnap.XG.data,'sig':siglayers}
                       ) * np.nan
vdz_snap = xr.DataArray(np.ones((len(dsnap.nmemb),
                                 len(dsnap.YG.sel(YG=slice(ystart-12**-1,yend+2*12**-1))),
                                 len(dsnap.XC),len(siglayers))),
                        dims=['nmemb','YG','XC','sig'],
                        coords={'nmemb':range(len(dsnap.nmemb),),
                                'YG':dsnap.YG.sel(YG=slice(ystart-12**-1,yend+2*12**-1)).data,
                                'XC':dsnap.XC.data,'sig':siglayers}
                       ) * np.nan

for tt in range(len(dsnap.nmemb)):
    tmp1 = xr.apply_ufunc(layers_numpy, 
                          dsnap.UVEL[tt].sel(YC=slice(ystart-12**-1,yend+2*12**-1)),
                          (rho_snapx[tt]-rhoConst).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                                                      ).chunk({'XG':xchunk}),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized',
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()
    
    tmp2 = xr.apply_ufunc(layers_numpy, 
                          dsnap.VVEL[tt].sel(YG=slice(ystart-12**-1,yend+2*12**-1)),
                          (rho_snapy[tt]-rhoConst).sel(YG=slice(ystart-12**-1,yend+2*12**-1)),
                          kwargs={'thetalayers':siglayers,'mapfact':mapfact,
                                  'mapindex':mapindex,'cellindex':cellindex,
                                  'drf_finer':drf_finer},
                          dask='parallelized',
                          input_core_dims=[['Z'],['Z']], 
                          output_core_dims=[['sig']],
                          output_dtypes=[float,], output_sizes={'sig':siglayers.size}
                         ).compute()


    udz_snap[tt] = tmp1
    vdz_snap[tt] = tmp2

    del tmp1, tmp2

    
# udz_snap = udz_snap.chunk({'nmemb':1,'sig':10,'XG':xchunk})
# vdz_snap = vdz_snap.chunk({'nmemb':1,'sig':10,'XC':xchunk})

In [22]:
dsave = zdz_snap.to_dataset(name='zdz')
dsave['zydz'] = zydz_snap
dsave['zxdz'] = zxdz_snap
dsave['dzeta'] = dzetaF_snap
dsave['dzetax'] = dzetaFx_snap
dsave['dzetay'] = dzetaFy_snap
# dsave['dzetaz'] = dzetaFz_snap
dsave.coords['sigp1'] = ('sigp1',sigp1layers)
dsave.to_netcdf(op.join(savedir,'%4d/Zs_membs_%010d.nc' % (1963,943920)))
dsave.close()

NameError: name 'grids' is not defined

In [25]:
dssig = zdz_snap.to_dataset(name='zdz')
# dssig['udz'] = udz
# dssig['vdz'] = vdz
dssig.coords['YG'] = ('YG',dsnap.YG.sel(YG=slice(ystart-12**-1,yend+2*12**-1)).data)
dssig.coords['XG'] = ('XG',dsnap.XG.data)
dssig.coords['sigp1'] = ('sigp1',sigp1layers)

buoyb = -grav * dssig.sig * rhoConst**-1
buoybp1 = -grav * dssig.sigp1 * rhoConst**-1

grids = Grid(dssig, periodic=['X'], 
             coords={'Z':{'center':'sig','outer':'sigp1'},
                     'Y':{'center':'YC','left':'YG'},
                     'X':{'center':'XC','left':'XG'}}
            )

In [8]:
dbdT = grids.diff(tdz*dzetaF_snap**-1,'Z',boundary='fill')**-1 * grids.diff(buoyb,'Z',boundary='fill')
dbds = grids.diff(sdz*dzetaF_snap**-1,'Z',boundary='fill')**-1 * grids.diff(buoyb,'Z',boundary='fill')
varpi = ((grids.interp(dbdT,'Z',boundary='fill') * diaTdz
          + grids.interp(dbds,'Z',boundary='fill') * diaSdz
         ) * dzetaF_snap**-1).compute()
dsave = vdz_snap.to_dataset(name='vdz')
dsave['udz'] = udz_snap
dsave['varpi'] = varpi
dsave['pxdz'] = pxdz_snap
dsave['pydz'] = pydz_snap
# dsave['tdz'] = tdz_snap
# dsave['sdz'] = sdz_snap
dsave.coords['sigp1'] = ('sigp1',sigp1layers)
dsave.to_netcdf(op.join(savedir,'%4d/Dyn_membs_%010d.nc' % (1963,943920)))
dsave.close() 

NameError: name 'tdz' is not defined

In [6]:
year = 1965
ds = open_mdsdataset(op.join(ddir,'qjamet/RUNS/ORAR/memb%02d/run%4d/ocn/' 
                             % (24,year)), 
                     grid_dir=op.join(ddir,'grid_chaO/gridMIT_update1/'),
                     iters=1103760,
                     prefix=['diag_ocnSnap'], delta_t=2e2,
                    ).sel(YC=slice(yNstart-12**-1,yend+2*12**-1),
                          YG=slice(yNstart-12**-1,yend+2*12**-1)
                         ).chunk({'XC':xchunk,'XG':xchunk,
                                  'YC':ychunk,'YG':ychunk})
ds

<xarray.Dataset>
Dimensions:  (XC: 1000, XG: 1000, YC: 483, YG: 483, Z: 46, Zl: 46, Zp1: 47, Zu: 46, time: 1)
Coordinates:
    iter     (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * time     (time) float64 2.208e+08
  * XC       (XC) >f4 262.00003 262.0861 262.17215 ... 347.88644 347.9725
  * YC       (YC) >f4 9.916701 10.000034 10.083367 ... 50.000034 50.08337
  * XG       (XG) >f4 261.957 262.04306 262.12912 ... 347.8434 347.92947
  * YG       (YG) >f4 9.958367 10.041701 10.125034 ... 50.041702 50.125034
  * Z        (Z) >f4 -3.0467727 -9.454049 -16.363966 ... -5624.9517 -5875.141
  * Zp1      (Zp1) >f4 0.0 -6.0935454 -12.814552 ... -5749.9097 -6000.3726
  * Zu       (Zu) >f4 -6.0935454 -12.814552 -19.91338 ... -5749.9097 -6000.3726
  * Zl       (Zl) >f4 0.0 -6.0935454 -12.814552 ... -5499.9937 -5749.9097
    rA       (YC, XC) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    dxG      (YG, XC) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    dyG      (YC, XG) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    Depth    (YC, XC) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    rAz      (YG, XG) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    dxC      (YC, XG) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    dyC      (YG, XC) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    rAw      (YC, XG) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    rAs      (YG, XC) >f4 dask.array<chunksize=(400, 250), meta=np.ndarray>
    drC      (Zp1) >f4 dask.array<chunksize=(47,), meta=np.ndarray>
    drF      (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    PHrefC   (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    PHrefF   (Zp1) >f4 dask.array<chunksize=(47,), meta=np.ndarray>
    hFacC    (Z, YC, XC) >f4 dask.array<chunksize=(46, 400, 250), meta=np.ndarray>
    hFacW    (Z, YC, XG) >f4 dask.array<chunksize=(46, 400, 250), meta=np.ndarray>
    hFacS    (Z, YG, XC) >f4 dask.array<chunksize=(46, 400, 250), meta=np.ndarray>
    maskC    (Z, YC, XC) bool dask.array<chunksize=(46, 400, 250), meta=np.ndarray>
    maskW    (Z, YC, XG) bool dask.array<chunksize=(46, 400, 250), meta=np.ndarray>
    maskS    (Z, YG, XC) bool dask.array<chunksize=(46, 400, 250), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(400, 250), meta=np.ndarray>
    maskInW  (YC, XG) bool dask.array<chunksize=(400, 250), meta=np.ndarray>
    maskInS  (YG, XC) bool dask.array<chunksize=(400, 250), meta=np.ndarray>
Data variables:
    THETA    (time, Z, YC, XC) >f4 dask.array<chunksize=(1, 46, 400, 250), meta=np.ndarray>
    SALT     (time, Z, YC, XC) >f4 dask.array<chunksize=(1, 46, 400, 250), meta=np.ndarray>
    UVEL     (time, Z, YC, XG) >f4 dask.array<chunksize=(1, 46, 400, 250), meta=np.ndarray>
    VVEL     (time, Z, YG, XC) >f4 dask.array<chunksize=(1, 46, 400, 250), meta=np.ndarray>
    WVEL     (time, Zl, YC, XC) >f4 dask.array<chunksize=(1, 46, 400, 250), meta=np.ndarray>
    PHIHYD   (time, Z, YC, XC) >f4 dask.array<chunksize=(1, 46, 400, 250), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        netCDF wrapper of MITgcm MDS binary data
    source:       MITgcm
    history:      Created by calling `open_mdsdataset(grid_dir='/tank/chaocea...

In [7]:
nremap = 80
ystart = yNstart

# ntimes = np.arange(943920,943920+dnf,dnf,dtype=int)
# ntimes = np.arange(943920+dnf,1101600+dnf,dnf, dtype=int)
ntimes = np.arange(1218240+dnf,1259280+dnf,dnf, dtype=int)
# ntimes = range(1259280+dnf,1416960+dnf,dnf)
for itime in ntimes:
    
    ds1 = xr.open_dataset(op.join(savedir,'%4d/Zs_membs_parallel-%2d_%010d.nc' % (year,nremap,itime))
                         ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                               YG=slice(ystart-12**-1,yend+2*12**-1)
                              ).chunk({'nmemb':membchunk,'XC':xchunk,'XG':xchunk})
    ds2 = xr.open_dataset(op.join(savedir,'%4d/Dyn_membs_parallel-%2d_%010d.nc' % (year,nremap,itime))
                         ).sel(YC=slice(ystart-12**-1,yend+2*12**-1),
                               YG=slice(ystart-12**-1,yend+2*12**-1)
                              ).chunk({'nmemb':membchunk,'XC':xchunk,'XG':xchunk})
    ds3 = xr.open_dataset(op.join(savedir,'%4d/Varpidz-compressible_membs_parallel-%2d_%010d.nc' 
                                  % (year,nremap,itime))
                         ).sel(YC=slice(ystart-12**-1,yend+2*12**-1)
                              ).chunk({'nmemb':membchunk,'XC':xchunk})
    buoyb = -grav * ds1.sig * rhoConst**-1
    buoybp1 = -grav * ds1.sigp1 * rhoConst**-1

    grid = Grid(ds1, periodic=['X'], 
                coords={'Z':{'center':'sig','outer':'sigp1'},
                        'Y':{'center':'YC','left':'YG'},
                        'X':{'center':'XC','left':'XG'}}
               )
    
###########################    
    sigma = ds1.dzetaF * -grid.diff(buoybp1,'Z',boundary='fill')**-1
    sigmay = ds1.dzetaFy * -grid.diff(buoybp1,'Z',boundary='fill')**-1
    sigmax = ds1.dzetaFx * -grid.diff(buoybp1,'Z',boundary='fill')**-1

    zetab = (ds1.zdz*ds1.dzetaF**-1).mean('nmemb',skipna=True)
    zetaxb = (ds1.zxdz*ds1.dzetaFx**-1).mean('nmemb',skipna=True)
    zetayb = (ds1.zydz*ds1.dzetaFy**-1).mean('nmemb',skipna=True)
    
    sigmab = (ds1.dzetaF.mean('nmemb',skipna=True) 
              * -grid.diff(buoybp1,'Z',boundary='fill')**-1
             )
    sigmaxb = (ds1.dzetaFx.mean('nmemb',skipna=True) 
               * -grid.diff(buoybp1,'Z',boundary='fill')**-1
              )
    sigmayb = (ds1.dzetaFy.mean('nmemb',skipna=True) 
               * -grid.diff(buoybp1,'Z',boundary='fill')**-1
              )
    sigmazb = (ds1.dzetaFz.mean('nmemb',skipna=True) 
               * -grid.diff(buoybp1,'Z',boundary='fill')**-1
              )
    
    vhat = ds2.vdz.mean('nmemb',skipna=True) * ds1.dzetaFy.mean('nmemb',skipna=True)**-1
    uhat = ds2.udz.mean('nmemb',skipna=True) * ds1.dzetaFx.mean('nmemb',skipna=True)**-1
    varpihat = ds3.varpidz.mean('nmemb',skipna=True) * ds1.dzetaF.mean('nmemb',skipna=True)**-1

    vpp = ds2.vdz*ds1.dzetaFy**-1 - vhat
    upp = ds2.udz*ds1.dzetaFx**-1 - uhat
    varpipp = ds3.varpidz*ds1.dzetaF**-1 - varpihat

    zetap = (ds1.zdz*ds1.dzetaF**-1 - zetab) 
    zetayp = (ds1.zydz*ds1.dzetaFy**-1 - zetayb)
    zetaxp = (ds1.zxdz*ds1.dzetaFx**-1 - zetaxb)

    mbx = ds2.pxdz * ds1.dzetaFx**-1
    mby = ds2.pydz * ds1.dzetaFy**-1
    mbxp = mbx - mbx.mean('nmemb',skipna=True)
    mbyp = mby - mby.mean('nmemb',skipna=True)
########################### 
    tmp00 = ((upp**2*sigmax)
             + .5*(zetaxp**2)
            ).mean('nmemb',skipna=True)
    tmp01 = (grid.interp(grid.interp(vpp,'X'),'Y',boundary='fill')
             * upp*sigmax).mean('nmemb',skipna=True)
    tmp10 = (grid.interp(grid.interp(upp,'X'),'Y',boundary='fill') 
             * vpp*sigmay).mean('nmemb',skipna=True)
    tmp11 = ((vpp**2*sigmay) 
             + .5*(zetayp**2)
            ).mean('nmemb',skipna=True)
    tmp20 = ((varpipp*grid.interp(upp,'X')*sigma)
             + grid.interp(mbxp*zetaxp,'X')
            ).mean('nmemb',skipna=True)
    tmp21 = ((varpipp*grid.interp(vpp,'Y',boundary='fill')*sigma)
             + grid.interp(mbyp*zetayp,'Y',boundary='fill')
            ).mean('nmemb',skipna=True)
    tmp00_0 = (upp**2*sigmax).mean('nmemb',skipna=True)
    tmp00_1 = .5*(zetaxp**2).mean('nmemb',skipna=True)
    tmp11_0 = (vpp**2*sigmay).mean('nmemb',skipna=True)
    tmp11_1 = .5*(zetayp**2).mean('nmemb',skipna=True)
###########################    
    start = time.time()
    e00 = (tmp00 * sigmaxb**-1).compute()
    e01 = (tmp01 * sigmaxb**-1).compute()
    e10 = (tmp10 * sigmayb**-1).compute()
    e11 = (tmp11 * sigmayb**-1).compute()
    e20 = (tmp20 * sigmab**-1).compute()
    e21 = (tmp21 * sigmab**-1).compute()
    dsave = e00.to_dataset(name='e00')
    dsave['e01'] = e01
    dsave['e10'] = e10
    dsave['e11'] = e11
    dsave['e20'] = e20
    dsave['e21'] = e21
    dsave.coords['YG'] = ('YG',ds1.YG.data)
    dsave.coords['XG'] = ('XG',ds1.XG.data)
    dsave.coords['sigp1'] = ('sigp1',ds1.sigp1)
    dsave.to_netcdf(op.join(savedir,'%4d/E-P_flux_parallel-%2d_%010d.nc' 
                            % (year,nremap,itime)), mode='w')
    dsave.close()
    
    del e00, e01, e10, e11, e20, e21
###########################  
    e00_x = ((grid.diff(tmp00_0*ds.dyG,'X') * ds.rA**-1
              + grid.diff(tmp00_1,'X') * grid.interp(ds.dxG,'Y',boundary='fill')**-1
             ) * sigmab**-1).compute()
    e10_y = (grid.diff(tmp10*ds.dxG,'Y',boundary='fill') * ds.rA**-1 
             * sigmab**-1).compute()
    e20_b = (grid.interp(grid.diff(tmp20,'Z',boundary='fill') 
                         * grid.diff(buoyb,'Z',boundary='fill')**-1,
                         'Z',boundary='fill')
             * sigmab**-1).compute()
    e01_x = (grid.diff(tmp01*ds.dyG,'X') * ds.rA**-1 
             * sigmab**-1).compute()
    e11_y = ((grid.diff(tmp11_0*ds.dxG,'Y',boundary='fill') * ds.rA**-1
              + grid.diff(tmp11_1,'Y',boundary='fill') * grid.interp(ds.dyG,'X')**-1
             ) * sigmab**-1).compute()
    e21_b = (grid.interp(grid.diff(tmp21,'Z',boundary='fill') 
                         * grid.diff(buoyb,'Z',boundary='fill')**-1,
                         'Z',boundary='fill') 
             * sigmab**-1).compute()
    dsave = e00_x.to_dataset(name='e00x')
    dsave['e01x'] = e01_x
    dsave['e10y'] = e10_y
    dsave['e11y'] = e11_y
    dsave['e20b'] = e20_b
    dsave['e21b'] = e21_b
    dsave.coords['YG'] = ('YG',ds1.YG.data)
    dsave.coords['XG'] = ('XG',ds1.XG.data)
    dsave.coords['sigp1'] = ('sigp1',ds1.sigp1)
    dsave.to_netcdf(op.join(savedir,'%4d/divE-P_flux_parallel-%2d_%010d.nc' 
                            % (year,nremap,itime)), mode='w')
    dsave.close()
    ds1.close()
    ds2.close()
    ds3.close()
    
    end = time.time()
    print("Lapse time:", end-start)
    
    del e00_x, e01_x, e10_y, e11_y, e20_b, e21_b
    print(itime)
    
# client.restart()

Lapse time: 474.65183305740356
1220400
Lapse time: 583.6783564090729
1222560
Lapse time: 715.3496987819672
1224720
Lapse time: 592.6705710887909
1226880
Lapse time: 685.9129354953766
1229040
Lapse time: 817.8308413028717
1231200
Lapse time: 704.6718909740448
1233360
Lapse time: 755.8726243972778
1235520
Lapse time: 817.1261785030365
1237680
Lapse time: 657.1469111442566
1239840
Lapse time: 581.4922668933868
1242000
Lapse time: 684.6812279224396
1244160
Lapse time: 695.3806383609772
1246320
Lapse time: 577.4643635749817
1248480
Lapse time: 595.4751250743866
1250640
Lapse time: 641.4541239738464
1252800
Lapse time: 705.252444267273
1254960
Lapse time: 647.4891600608826
1257120
Lapse time: 745.5801663398743
1259280


Client Scheduler: tcp://127.0.0.1:48707 Dashboard: http://127.0.0.1:44931/status,Cluster Workers: 6 Cores: 30 Memory: 127.00 GB
